# Breaking RSA Encryption Scheme using Shor's algorithm

# <a id='index'>Index</a>

+ <a href='#key'>1. Generation of Keys</a>
+ <a href='#encrypt'>2. Encrypting plain_text</a>
+ <a href='#decrypt'>3. Decrypting cipher_text</a>
+ <a href='#shor'>***4. Shor's function to break RSA***</a>
+ <a href='#aux'>5. Auxiliary functions</a>
+ <a href='#implementation'>6. Implementation</a>
    + <a href='#6.1'>6.1 Generating Keys</a>
    + <a href='#6.2'>6.2 Generating plain_text</a>
    + <a href='#6.3'>6.3 Encrypting plain_text to generate cipher_text</a>
    + <a href='#6.4'>***6.4 Attacking RSA and changing sender's message***</a>
    + <a href='#6.5'>6.5 Decrypting cipher_text to generate plain_text</a>
    + <a href='#6.6'>6.6 Checking if decrypted plain_text is same as original plain_text</a>
+ <a href='#final'>7. Final Remarks</a>

In [281]:
import numpy as np
import math
import gmpy2
from gmpy2 import powmod,mpz,isqrt,invert
from qiskit.aqua.algorithms import Shor
from qiskit.aqua import QuantumInstance
from qiskit import Aer,execute,QuantumCircuit
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from qiskit import IBMQ, execute

# <a id='key'>1. Generation of Keys</a>
<a href='#index'>Go back to the top</a>

***Key Generation In RSA :***

+ Choose 2 primes p,q using gmpy
+ n = p*q and $ \phi (n) = (p-1)*(q-1) $
+ Choose $ e $ in $ [2,\phi(n)-2] $ such that $ e $ and $ \phi(n) $ are coprimes i.e their gcd is 1
+ Choose $ d = e^{-1} mod(\phi(n)) $
+ Return ***Public keys :*** $ (n,e) $
+ Return ***Private key :*** $ (d) $

In [282]:
def generate_keys():
    # prime number of 3 digits i.e 7 bits
    random1 = np.random.randint(3,40)
    random2 = np.random.randint(3,40)
    p = int(gmpy2.next_prime(random1))
    q = int(gmpy2.next_prime(random2))
    n = p*q
    while (n<100 or n>127):
        random1 = np.random.randint(3,40)
        random2 = np.random.randint(3,40)
        p = int(gmpy2.next_prime(random1))
        q = int(gmpy2.next_prime(random2))
        n = p*q
    phi = (p-1)*(q-1)
    e = 2
    while True:
        if gmpy2.gcd(phi,e) != 1:
            e = e + 1
        else :
            break
    d = gmpy2.invert(e,phi)
    return n,e,d

# <a id='encrypt'>2. Encrypting plain_text blocks</a>
<a href='#index'>Go back to the top</a>

***Encyption in RSA :***

***Input :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

***Output :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

where

$ c_{i} = p_{i}^e mod (n) $


In [283]:
def encrypt(plain_text_blocks,public_keys):
    cipher_text_blocks = []
    n,e = public_keys
    for plain_text in plain_text_blocks:
        cipher_text = (gmpy2.powmod(plain_text,e,n))
        cipher_text_blocks.append(cipher_text)
    return cipher_text_blocks

# <a id='decrypt'>3. Decrypting cipher_text</a>
<a href='#index'>Go back to the top</a>

***Decryption in RSA :***

***Input :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

***Output :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

$ p_{i} = c_{i}^d mod (n) $

In [284]:
def decrypt(cipher_text_blocks,secret_key,public_keys):
    n,e = public_keys
    d = secret_key
    decypted_plain_text_blocks = []
    for cipher_text in cipher_text_blocks:
        plain_text = (gmpy2.powmod(cipher_text,d,n))
        decypted_plain_text_blocks.append(plain_text)
    return decypted_plain_text_blocks

# <a id='shor'>4. Shor's function to break RSA</a>

+ Shor's algorithm requires $ 4*n_{bits} + 4 $ qubits to factorize $ n_{bits} $ bit number
+ We are using ***ibm_qasm_simulator*** which has upper limit of $ 32 $ qubits.
+ $ 4n_{bits} + 4 \leq 32 $ solving this we get $ 4n_{bits} \leq 7 $
+ So we can use maximum of 7 bit binary number $ n $ i.e $ n \leq 127 $
+ Since we are encrypting digits from $ 0-99 $ to encode all alpha-numeric characters along with special characters,we want $ n \gt 100 $
+ $ \therefore 100 \lt n \leq 127$ 

In [285]:
def get_factors(public_keys):
    n,e = public_keys
    # backend = Aer.get_backend('qasm_simulator')
    provider = IBMQ.load_account()
    backend = provider.get_backend('ibmq_qasm_simulator')
    quantum_instance = QuantumInstance(backend,shots=2500)
    find_factors = Shor(n,a=2,quantum_instance=quantum_instance)
    factors = Shor.run(find_factors)
    p = ((factors['factors'])[0])[0]
    q = ((factors['factors'])[0])[1]
    print('Factors of',n,'are :',p,q)
    
    return p,q

# <a id='aux'>5. Auxiliary functions</a>
<a href='#index'>Go back to the top</a>

In [286]:
# taken in 'Hello World!!!' returns ['Hello World!','!!']
def get_blocks(PT,block_size):
    blocks = []
    i = 0
    while i<len(PT):
        temp_str=''
        if i+block_size-1 < len(PT):
            temp_str=temp_str+PT[i:i+block_size]
        else :
            temp_str=temp_str+PT[i::]
        blocks.append(temp_str)
        i=i+block_size
    return blocks
        
# covert plain_text block from characters to the numbers
def format_plain_text(PT):
    plain_text_blocks = []
    for block in PT:
        plain_text = 0
        for i in range(len(block)):
            # for 'd'
            if ord(block[i]) == 100:
                plain_text = plain_text*100 + 28
            # between (101,127)
            elif ord(block[i])>100:
                plain_text = plain_text*100 + (ord(block[i])-100)
            else :
                plain_text = plain_text*100 + (ord(block[i]))
        plain_text_blocks.append(plain_text)
    return plain_text_blocks

# convert numeric decypted_plain_text_blocks into a single plain text of characters
def format_decrypted_plain_text(decypted_plain_text_blocks):
    plain_text_blocks = []
    for dc_pt in decypted_plain_text_blocks:
        plain_text = ''
        temp = dc_pt
        # for 'd' temp = 28
        while temp > 0:
            if temp%100 == 28:
                plain_text = plain_text + 'd'
            elif (temp%100) in range(0,27):
                plain_text = plain_text + chr((temp%100)+100)
            else :
                plain_text = plain_text + chr((temp%100))
            temp = temp//100
        plain_text = plain_text[::-1] 
        plain_text_blocks.append(plain_text)
    final_plain_text = ''
    for plain_text_block in plain_text_blocks:
        final_plain_text = final_plain_text + plain_text_block
    return final_plain_text

# <a id='implementation'>6. Implementation</a>
<a href='#index'>Go back to the top</a>

### <a id='6.1'>6.1 Generating Keys</a>

In [287]:
n,e,d = generate_keys()

In [288]:
public_keys = (n,e)
secret_key = d
print("\nPublic Key :")
print('n :',n)
print('e :',e)
print("Secret Key :\nd :",d)


Public Key :
n : 119
e : 5
Secret Key :
d : 77


### <a id='6.2'>6.2 Generating plain_text</a>

In [297]:
PT = input("\nEnter Plain Text to encrypt : ")

original_plain_text = PT


Enter Plain Text to encrypt : Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


In [290]:
block_size = 1
PT = get_blocks(PT,block_size)
print('\nPlain Text after converting to blocks',PT)


Plain Text after converting to blocks ['S', 'c', 'h', 'r', 'o', 'd', 'i', 'n', 'g', 'e', 'r', "'", 's', ' ', 'c', 'a', 't', ' ', 'i', 's', ' ', 'd', 'e', 'a', 'd', ' ', 'a', 'n', 'd', ' ', 'a', 'l', 'i', 'v', 'e', ' ', 'a', 't', ' ', 't', 'h', 'e', ' ', 's', 'a', 'm', 'e', ' ', 't', 'i', 'm', 'e', '.', ' ', 'T', 'h', 'i', 's', ' ', 'c', 'a', 't', ' ', 'r', 'e', 'p', 'r', 'e', 's', 'e', 'n', 't', 's', ' ', 't', 'h', 'e', ' ', '(', '|', 'p', 's', 'i', '_', 'c', 'a', 't', '>', '=', '0', '.', '5', '*', '|', 'p', 's', 'i', '_', 'c', 'a', 't', '_', 'a', 'l', 'i', 'v', 'e', '>', ' ', '+', ' ', '0', '.', '5', '*', '|', 'p', 's', 'i', '_', 'c', 'a', 't', '_', 'd', 'e', 'a', 'd', '>', ')', ' ', 'i', '.', 'e', ' ', 's', 'u', 'p', 'e', 'r', 'p', 'o', 's', 'i', 't', 'i', 'o', 'n', ' ', 'o', 'f', ' ', 'w', 'a', 'v', 'e', '-', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n', 's', ' ', 'r', 'e', 'p', 'r', 'e', 's', 'e', 'n', 't', 'i', 'n', 'g', ' ', 'c', 'a', 't', ' ', 'b', 'e', 'i', 'n', 'g', ' ', 'd', 'e', 

In [291]:
plain_text_blocks = format_plain_text(PT)
print('\nPlain text blocks after formatting to numbers:',plain_text_blocks)


Plain text blocks after formatting to numbers: [83, 99, 4, 14, 11, 28, 5, 10, 3, 1, 14, 39, 15, 32, 99, 97, 16, 32, 5, 15, 32, 28, 1, 97, 28, 32, 97, 10, 28, 32, 97, 8, 5, 18, 1, 32, 97, 16, 32, 16, 4, 1, 32, 15, 97, 9, 1, 32, 16, 5, 9, 1, 46, 32, 84, 4, 5, 15, 32, 99, 97, 16, 32, 14, 1, 12, 14, 1, 15, 1, 10, 16, 15, 32, 16, 4, 1, 32, 40, 24, 12, 15, 5, 95, 99, 97, 16, 62, 61, 48, 46, 53, 42, 24, 12, 15, 5, 95, 99, 97, 16, 95, 97, 8, 5, 18, 1, 62, 32, 43, 32, 48, 46, 53, 42, 24, 12, 15, 5, 95, 99, 97, 16, 95, 28, 1, 97, 28, 62, 41, 32, 5, 46, 1, 32, 15, 17, 12, 1, 14, 12, 11, 15, 5, 16, 5, 11, 10, 32, 11, 2, 32, 19, 97, 18, 1, 45, 2, 17, 10, 99, 16, 5, 11, 10, 15, 32, 14, 1, 12, 14, 1, 15, 1, 10, 16, 5, 10, 3, 32, 99, 97, 16, 32, 98, 1, 5, 10, 3, 32, 28, 1, 97, 28, 32, 97, 10, 28, 32, 97, 8, 5, 18, 1, 32, 97, 16, 32, 16, 4, 1, 32, 15, 97, 9, 1, 32, 16, 5, 9, 1, 46]


### <a id='6.3'>6.3 Encrypting plain_text to generate cipher_text</a>

In [292]:
cipher_text_blocks = encrypt(plain_text_blocks,public_keys)
print("\nCipher Text Blocks After RSA encryption :",cipher_text_blocks)


Cipher Text Blocks After RSA encryption : [mpz(104), mpz(29), mpz(72), mpz(63), mpz(44), mpz(112), mpz(31), mpz(40), mpz(5), mpz(1), mpz(63), mpz(65), mpz(36), mpz(2), mpz(29), mpz(20), mpz(67), mpz(2), mpz(31), mpz(36), mpz(2), mpz(112), mpz(1), mpz(20), mpz(112), mpz(2), mpz(20), mpz(40), mpz(112), mpz(2), mpz(20), mpz(43), mpz(31), mpz(86), mpz(1), mpz(2), mpz(20), mpz(67), mpz(2), mpz(67), mpz(72), mpz(1), mpz(2), mpz(36), mpz(20), mpz(25), mpz(1), mpz(2), mpz(67), mpz(31), mpz(25), mpz(1), mpz(37), mpz(2), mpz(84), mpz(72), mpz(31), mpz(36), mpz(2), mpz(29), mpz(20), mpz(67), mpz(2), mpz(63), mpz(1), mpz(3), mpz(63), mpz(1), mpz(36), mpz(1), mpz(40), mpz(67), mpz(36), mpz(2), mpz(67), mpz(72), mpz(1), mpz(2), mpz(24), mpz(96), mpz(3), mpz(36), mpz(31), mpz(23), mpz(29), mpz(20), mpz(67), mpz(27), mpz(108), mpz(97), mpz(37), mpz(100), mpz(77), mpz(96), mpz(3), mpz(36), mpz(31), mpz(23), mpz(29), mpz(20), mpz(67), mpz(23), mpz(20), mpz(43), mpz(31), mpz(86), mpz(1), mpz(27), mpz(2)

### <a id='6.4'>6.4 Attacking RSA and changing sender's message</a>

In [293]:
p,q = get_factors(public_keys)
phi = (p-1)*(q-1)
broken_d = gmpy2.invert(e,phi)
compromised_PT = decrypt(cipher_text_blocks,broken_d,public_keys)
compromised_PT = format_decrypted_plain_text(compromised_PT)
compromised_PT = '!!!Your message has been attacked!!! ' + compromised_PT
compromised_PT = get_blocks(compromised_PT,block_size) 
compromised_PT = format_plain_text(compromised_PT)
compromised_CT = encrypt(compromised_PT,public_keys)

cipher_text_blocks = compromised_CT

C:\ProgramData\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2020-12-05 02:48:28,171: Credentials are already in use. The existing account in the session will be replaced.
The skip Qobj validation does not work for IBMQ provider. Disable it.


Factors of 119 are : 7 17


### <a id='6.5'>6.5 Decrypting cipher_text to generate plain_text</a>

In [294]:
decypted_plain_text_blocks = decrypt(cipher_text_blocks,secret_key,public_keys)
print("\nPlain Text blocks after decryption of Cipher Text blocks :",decypted_plain_text_blocks)


Plain Text blocks after decryption of Cipher Text blocks : [mpz(33), mpz(33), mpz(33), mpz(89), mpz(11), mpz(17), mpz(14), mpz(32), mpz(9), mpz(1), mpz(15), mpz(15), mpz(97), mpz(3), mpz(1), mpz(32), mpz(4), mpz(97), mpz(15), mpz(32), mpz(98), mpz(1), mpz(1), mpz(10), mpz(32), mpz(97), mpz(16), mpz(16), mpz(97), mpz(99), mpz(7), mpz(1), mpz(28), mpz(33), mpz(33), mpz(33), mpz(32), mpz(83), mpz(99), mpz(4), mpz(14), mpz(11), mpz(28), mpz(5), mpz(10), mpz(3), mpz(1), mpz(14), mpz(39), mpz(15), mpz(32), mpz(99), mpz(97), mpz(16), mpz(32), mpz(5), mpz(15), mpz(32), mpz(28), mpz(1), mpz(97), mpz(28), mpz(32), mpz(97), mpz(10), mpz(28), mpz(32), mpz(97), mpz(8), mpz(5), mpz(18), mpz(1), mpz(32), mpz(97), mpz(16), mpz(32), mpz(16), mpz(4), mpz(1), mpz(32), mpz(15), mpz(97), mpz(9), mpz(1), mpz(32), mpz(16), mpz(5), mpz(9), mpz(1), mpz(46), mpz(32), mpz(84), mpz(4), mpz(5), mpz(15), mpz(32), mpz(99), mpz(97), mpz(16), mpz(32), mpz(14), mpz(1), mpz(12), mpz(14), mpz(1), mpz(15), mpz(1), mpz(10

In [295]:
plain_text_after_decryption = format_decrypted_plain_text(decypted_plain_text_blocks)
print("\nAfter decryption Plain Text :",plain_text_after_decryption)


After decryption Plain Text : !!!Your message has been attacked!!! Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


### <a id='6.6'>6.6 Checking if decrypted plain_text is same as original plain_text</a>

In [296]:
if (original_plain_text == plain_text_after_decryption):
    print("\nHurrayyy!!!\n\nDecrypted plain_text is same as original plain_text! :) ")
else :
    print('RSA was attacked!!! :(')

RSA was attacked!!! :(


# <a id='final'>7. Final Remarks</a>
<a href='#index'>Go back to the top</a>

+ ASCII values :
    + ASCII values from 0-31 are not alpha-numeric characters.
    + ASCII values from ***32-99 contain [A_Z,a,b,c] and special characters such as @,#,',",space,etc***
    + ASCII values from ***100-126 contain [d-z] which are 3 digit.***
    + Since we're ***not using ASCII values from 0-31, we can convert ASCII values of [d-z] to 2-digit number, by subtracting 100 from them.***
    + If we subtract 100 from ASCII values of 'd', which is 100, we get 00. ***d will be 00 which 0***, so 'd' is lost in the processing. 
    + Unused ASCII values we have now are [27,28,29,30,31], so we assign d=28. 
+ This program uses 3 digit prime number, so it can encrypt upto maximum of 1 characters at a time i.e 2 digits
+ So, to encrypt large plain text, ***RSA was implemented as a block cipher.***